In [15]:
import numpy as np
import json_tricks as json_tricks
import os

answer = {}
inputs = json_tricks.load('inputs.json')

FileNotFoundError: [Errno 2] No such file or directory: 'inputs.json'

At first I can not open THIS FILES AND I TOOK 30 MINS TO SOLVE IT. YOU KNOW WHAT CAUSE IT??? 

                      ,                      

This thing. YES JUST """" , """" this! OMG I'm crying, Coding is so fun HAHAHA //crying

# Matrix Product II

$$A_{5 \times 10} \  B_{10 \times 2}\ C_{2 \times 30}\ D_{30 \times 3}\ E_{3 \times 9}$$

1. What will be the shape of the resulting matrix?
2. How many multiplication of numbers are required at best?

In [ ]:
answer['task1'] = {
    '1': [5, 9],
    '2': 424
}


////SHORTNOTE////


Where is 424 come from??


So we use "Matrix Chain Order"


The thing that he taught us in class I REMEMBERED DID'T I???? [(((AB)C)D)E]



Actually we have to try a lot in this process like (((AB)C)D)E or (A(B(C(DE)))) or ((AB)((CD)E)) and so on. SO SINCERELY USE AI FOR THIS IS BEST WAY 



Let do for make sure I am understand


((AB)((CD)E))


Sol

 A*B = (N * M) * ( M * L) = N * M * L = 5 * 10 * 2 = 100


 C*D = (L * V) * (V * H) = L * V * H = 2 * 30 * 3 = 180


(C*D)*E = (L * V * H ) * ( H * U ) = L * H * U = 2 * 3 * 9 = 54


(A*B)*((C*D)*E) = (N * M * L ) * (L * V * H) * (H * U) = N * H * U = 5 * 2 * 9 = 90




100+180+54+90 = 424


# Numpy expression

Using Numpy, write a function that calculates the 
following expression:

$$\exp(A^T(B + 2C) + 3I) \mathbf x,$$

where $I$ is an identity matrix of the necessary shape.

In [ ]:
def numpy_expression(A, B, C, x):
    # Compute the expression: exp(A^T(B + 2C) + 3I) x
    inner = B + 2 * C
    M = A.T @ inner
    # The identity matrix should be square and match M's shape
    if M.shape[0] == M.shape[1]:
        M_plus_I = M + 3 * np.eye(M.shape[0])
    else:
        # If not square, only add to the diagonal part (broadcasting)
        M_plus_I = M + 3 * np.eye(*M.shape)
    expr = np.exp(M_plus_I)
    # Ensure x is the correct shape for multiplication
    result = expr @ x
    return result

In [ ]:
answer['task2'] = []
for one_input in inputs.get('task2', []):
    answer['task2'].append(numpy_expression(**one_input))

# Einstein's Rule

In *Tensor Algebra*, a direct generalization of the Linear Algebra to the case of $N$-dimentional tables called *tensors* (normal matrix), the Einstein's rule exists.

It works as follows: if you see a duplicating upper and lower index in the formula, that means, this index convolves.

For example, the following tensor expression, summation and matrix product are equivalent:

$$a_k^l b_l^m = \sum_{l=1}^L a_k^l b_l^m = AB$$

In this notation subscript means row index and superscript means column index.

<details>
<summary> Note </summary>

> [!NOTE]
> Also at some point it will be important to know that:
> * lower index represents a contravariant dimension of a
> tensor
> * upper index represents a covariant dimension 
> of tensor. But let us omit this part for now.

</details>

# Task

Calculate the following expression written using Einstein's 
rule:

$$a_k^m b_m^n c_n^o d_l^k$$

In [ ]:
def einsteins_rule(A, B, C, D):
    result = np.einsum('lk,km,mn,no->lo', D, A, B, C)
    return result


In [ ]:
answer['task3'] = []
if 'task3' in inputs:
    for one_input in inputs['task3']:
        answer['task3'].append(einsteins_rule(**one_input))


# Diagonal Matrix Product

You are given two square matrices: $A$ and $D$, where $A$ is a 
full matrix and $D$ is a diagonal matrix:

$$
A = \begin{bmatrix}
- & \mathbf a_1 & - \\
& \vdots & \\
- & \mathbf a_N & - \\
\end{bmatrix}
$$

$$
D = \textrm{diag}(d_1, d_2, \dots, d_N) = \begin{bmatrix}
d_1 & & & & \\
& d_2 & & & \\
& & d_3 & & \\
& & & \ddots & \\
& & & & d_N 
\end{bmatrix}
$$

Write a program to calculate the result of $DA$ and $AD$ in 
the fastest possible way.

In [ ]:
def diag_prod_DA(A, D):
    # D is a diagonal matrix, extract the diagonal
    d = np.diag(D)
    DA = d[:, np.newaxis] * A
    return DA

def diag_prod_AD(A, D):
    # D is a diagonal matrix, extract the diagonal
    d = np.diag(D)
    AD = A * d
    return AD

In [ ]:
answer['task4_1'] = []
answer['task4_2'] = []
if 'task4' in inputs:
    for one_input in inputs['task4']:
        answer['task4_1'].append(diag_prod_DA(**one_input))
        answer['task4_2'].append(diag_prod_AD(**one_input))

# Sparse Matrix Product

You are given two matrices of the same shape: $A$ and $B$. Matrix $A$ is full
and is given in the form of `numpy.ndarray`.

The second matrix $B$ is **sparse**. That means that the 
majority of the items are equal to $0$ except for $M$. This matrix is given
as a set of non-zero elements of this matrix in form of $3 \times M$ `numpy.ndarray` as row-column-value tuple (COO sparse matrix form):

$$
\begin{bmatrix}
r_1 & c_1 & v_1 \\
r_2 & c_2 & v_2 \\
& \vdots & \\
r_M & c_M & v_M \\
\end{bmatrix}
$$

If in this struct two items correspond to the same location, consider the latter is correct.

Write the most efficient program that calculates $AB$.

Also return the ratio between the number of multiplication operations that are needed to calculate the sparse product and the number of operations for full product.

In [ ]:
def sparse_matrix_product(A, B_sparse):
    N, K = A.shape
    # Ensure B_sparse is shape (M, 3)
    if B_sparse.shape[0] == 3:
        B_sparse = B_sparse.T
    # Reconstruct B as a full matrix, using the last value for each (r, c)
    M = int(B_sparse[:, 1].max()) + 1  # number of columns in B
    B = np.zeros((K, M), dtype=float)
    for r, c, v in B_sparse:
        r = int(r)
        c = int(c)
        v = float(v)
        B[r, c] = v  # overwrite, not accumulate

    result = A @ B
    # Count only the multiplications for nonzero elements in B
    mult_ops = int(np.count_nonzero(B)) * N
    full_ops = N * K * M
    return result, float(mult_ops) / float(full_ops)

In [ ]:
answer['task5'] = []
if 'task5' in inputs:
    for one_input in inputs['task5']:
        result, ratio = sparse_matrix_product(**one_input)
        answer['task5'].append([result, ratio])

In [ ]:
json_tricks.dump(answer, '.answer.json')

'{"task1": {"1": [5, 9], "2": 424}, "task2": [], "task3": [], "task4_1": [], "task4_2": [], "task5": []}'